In [1]:
# from src.crawler import Crawler

In [ ]:
CRWALING = True         # 크롤링
DB_BULK_INSERT = True   # 크롤링 데이터 DB INSERT

TEST_SIZE = 1           # 크롤링 공고 테스트용 사이트 설정되어 있는 만큼 진행됨 기본값 -1 = 전체 진행
batch_id = ''           # 중간 진입위한 BATCH_ID 넣으면 

# 중간에 끊긴 경우: 아래 주석 해제하고 실행하면 마지막 batch_id로 이어서 진행
# from src.database import DataBaseHandler
# with DataBaseHandler() as db:
#     result = db.execute_query("SELECT batch_id FROM annc_lh_temp ORDER BY batch_start_dttm DESC LIMIT 1")
#     if result:
#         batch_id = str(result[0]['batch_id'])
#         print(f"마지막 batch_id: {batch_id}")

if batch_id:
    CRWALING = False
    DB_BULK_INSERT = False

마지막 batch_id: b3885313-b5cb-4066-8c0d-8d350c8782a8


In [3]:
import os
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

True

## 1. 테이블 존재 여부 확인

## 0. DB 초기화 (선택사항)
기존 데이터를 모두 삭제하고 테이블을 재생성합니다.  
**주의: 모든 공고, 파일, 청크 데이터가 삭제됩니다!**

In [4]:
# # DB 초기화 실행 (필요시에만 주석 해제 후 실행)
# from src.database.initializing import Initializing

# init = Initializing()
# init.set_default_tables(drop=True)  # 기존 테이블 삭제 후 재생성

In [5]:
from src.database import DataBaseHandler

tables = ("annc_all","annc_files","annc_lh_temp","doc_chunks")

cases = ','.join([f"""sum(case when table_name in ('{table}') then 1 else 0 end) as {table}""" for table in tables])

query = f"""
SELECT 
{cases}
FROM information_schema.tables
WHERE table_schema = 'public'
AND table_name = any(%s)
"""

with DataBaseHandler() as db:
    result = db.execute_query(query, (list(tables),))

zero_count = 0
for val in result[0].items():
    table_name = val[0]
    table_count = val[1]
    print(f"{val[0]}: {'✅' if val[1] > 0 else '❌'}")
    if table_count == 0:
        zero_count += 1
if zero_count > 0:
    print("테이블 이상 상태 확인 필요")
    raise

annc_all: ✅
annc_files: ✅
annc_lh_temp: ✅
doc_chunks: ✅


## 2. 크롤링

In [6]:
# 크롤링 본 로직

from src.crawler.lh import LH


lh_crwaler = LH()

if CRWALING:
    df_all_annc = lh_crwaler.crawl_lh_notices_all_data('접수중')
    df_all_annc += lh_crwaler.crawl_lh_notices_all_data('공고중')
    # df_all_annc += lh_crwaler.crawl_lh_notices_all_data('정정공고중')
else:
    df_all_annc = []
    print("크롤링 x")

크롤링 x


In [7]:
# 크롤링 된 데이터 중복제거

len_before = len(df_all_annc)
unique_tuples = set(tuple(sorted(d.items())) for d in df_all_annc)
df_all_annc = [dict(t) for t in unique_tuples]
len_after = len(df_all_annc)

print(f'중복제거 {len_before} ➡️ {len_after}')

중복제거 0 ➡️ 0


In [8]:
len_before = len(df_all_annc)
df_all_annc = [annc for annc in df_all_annc if annc['annc_type'] in ('매입','임대')]
len_after = len(df_all_annc)

print(f'기타항목 제거 {len_before} ➡️ {len_after}')

기타항목 제거 0 ➡️ 0


In [9]:
len_before = len(df_all_annc)
# df_all_annc = [annc for annc in df_all_annc if annc['annc_region'] in ('서울특별시','경기도')]
df_all_annc = [annc for annc in df_all_annc]
len_after = len(df_all_annc)

print(f'전지역 {len_before} ➡️ {len_after}')

전지역 0 ➡️ 0


In [10]:
df_all_annc

[]

# 3. 대상 DB INSERT - BATCH ID 발행

In [11]:
from src.database.repository import (
    AnncLhRepository, AnncQrRepository,
    AnncAllRepository, AnncFileRepository,
    DocChunkRepository,
)

lh_repo = AnncLhRepository()
# qr_repo = AnncQrRepository()
all_repo = AnncAllRepository()
file_repo = AnncFileRepository()
dc_repo = DocChunkRepository()

if DB_BULK_INSERT: 
    batch_id = lh_repo.bulk_insert_announcements(df_all_annc)
else:
    print("크롤링 데이터 삽입 x")

if not batch_id:
    raise "batch_id가 없으면 진행 불가."


크롤링 데이터 삽입 x


In [12]:
annc_list_lh = AnncQrRepository().get_announcements_merge_target(batch_id, annc_status=['공고중','접수중'])

print(f'{len(annc_list_lh)}건 조회 완료')

13건 조회 완료


# 4. 파일 등록

In [13]:
from datetime import datetime, timedelta
from src.parser import parse_pdf
from src.chunker import create_chunks_from_elements
from src.embedder import embed_chunks
import json

def process(row_lh, corp_cd):
    """
    공고 리스트
    """
    time_laps = []

    title_now = lambda title: (title, datetime.now())

    # 1. 배치 테이블 상태 변경 -> 시작
    lh_repo.update_announcements('START', row_lh['batch_id'], row_lh['batch_seq'])
    time_laps.append(title_now(f"배치 시작 - {row_lh['annc_title']}"))

    # 2. 공고 닫기 처리
    row_lh['corp_cd'] = corp_cd
    row_lh['service_status'] = 'CLOSE'
    merge_result = all_repo.merge_announcements([row_lh,]) # 원래 다건을 위한것
    time_laps.append(title_now(f"공고 닫기 처리"))

    if not merge_result:
        raise Exception("머지된 행 없음")
    
    merge_result = merge_result[0]
    annc_id = merge_result['annc_id']

    # 3. 파일 조회
    file_list = lh_crwaler.get_file_list(row_lh)
    time_laps.append(title_now(f"파일 조회"))

    if not file_list:
        raise Exception("파일 없음")
    
    # 4. 파일, 청크 벡터 초기화
    dc_repo.delete_chunks_by_annc_id(annc_id)
    file_repo.delete_files_by_annc_id(annc_id)
    # print(f"{get_elapsed_time()} {batch_msg} - 파일 초기화")
    time_laps.append(title_now(f"파일 기록 삭제 처리"))

    for idx_file, file_info in enumerate(file_list):
        time_laps.append(title_now(f"파일 처리 시작 ({idx_file+1}/{len(file_list)})"))

        annc_file = {}

        annc_file['annc_id'] = annc_id
        annc_file['file_name'] = file_info['cmnAhflNm']
        annc_file['file_type'] = file_info['slPanAhflDsCdNm']
        annc_file['file_ext'] = 'pdf'

        # 파일 다운
        file_path, annc_file = lh_crwaler.down_file(file_info['cmnAhflSn'], annc_file)
        time_laps.append(title_now(f"파일 다운로드"))

        # 파일 DB 등록
        inserted_file_info = file_repo.bulk_insert_files([annc_file])[0]
        file_id, file_name = inserted_file_info['file_id'], inserted_file_info['file_name']
        time_laps.append(title_now(f"파일 정보 DB 기록"))

        # 파일 엘리먼트 구성
        parsed = parse_pdf(file_path, annc_id)
        time_laps.append(title_now(f"파일 -> Markdown"))
        # test_elements = parsed.elements[:10]

        chunks = create_chunks_from_elements(parsed.elements, annc_id)
        time_laps.append(title_now(f"Markdown -> 청크"))

        embed_chunks(chunks)
        time_laps.append(title_now(f"청크 -> 임베딩"))

        chunk_dto = [{
            'file_id': file_id,
            'annc_id': annc_id,
            'chunk_type': c.element_type,#get('element_type','text'),
            'chunk_text': c.text, #get('text',''),
            'page_num': c.page_number,
            'embedding': c.embedding,
            'metadata': json.dumps(c.metadata)  # dict를 JSON 문자열로 변환
        } for c in chunks]

        
        dc_repo.bulk_insert_chunks(chunk_dto)
        time_laps.append(title_now(f"임베딩 정보 기록"))

        # 사용한 파일 삭제
        if os.path.exists(file_path):
            os.remove(file_path)
        time_laps.append(title_now(f"파일 삭제"))

        # return chunks
    lh_repo.update_announcements('COMPLETE', row_lh['batch_id'], row_lh['batch_seq'])
    time_laps.append(title_now(f"배치 종료"))

    # print(parsed)
    row_lh['service_status'] = 'OPEN'
    merge_result = all_repo.merge_announcements([row_lh,]) # 원래 다건을 위한것
    time_laps.append(title_now(f"배치 완료 처리"))

    return time_laps

In [14]:
def print_time_laps(time_laps):
    """
    process()의 반환값(time_laps)을 받아
    - 직전 단계 대비 소요시간
    - 시작 이후 총 소요시간
    을 함께 출력한다.
    """
    if not time_laps:
        print("시간 정보가 없습니다.")
        return

    start_time = time_laps[0][1]

    for i, (title, t) in enumerate(time_laps):
        if i == 0:
            diff_prev = timedelta(0)
        else:
            diff_prev = t - time_laps[i-1][1]

        diff_total = t - start_time

        print(f"[{i+1:02d}] {title}")
        print(f"  ├ 전 단계 대비: {diff_prev}")
        print(f"  └ 전체 경과   : {diff_total}")

## 실행 및 타임 랩스

In [15]:
for row in annc_list_lh:
    time_laps = process(row,"LH")

    print_time_laps(time_laps)


/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id fcc968ec-f635-4565-8db3-92ee566f5fdf
[01] 배치 시작 - 경북청도 행복주택 입주자 최초모집 공고
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.011811
  └ 전체 경과   : 0:00:00.011811
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.128292
  └ 전체 경과   : 0:00:00.140103
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.028872
  └ 전체 경과   : 0:00:00.168975
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000012
  └ 전체 경과   : 0:00:00.168987
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.371233
  └ 전체 경과   : 0:00:00.540220
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.023081
  └ 전체 경과   : 0:00:00.563301
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:00:25.645667
  └ 전체 경과   : 0:00:26.208968
[09] Markdown -> 청크
  ├ 전 단계 대비: 0:00:00.045940
  └ 전체 경과   : 0:00:26.254908
[10] 청크 -> 임베딩
  ├ 전 단계 대비: 0:00:17.581631
  └ 전체 경과   : 0:00:43.836539
[11] 임베딩 정보 기록
  ├ 전 단계 대비: 0:00:00.639574
  └ 전체 경과   : 0:00:44.476113
[12] 파일 삭제
  ├ 전 단계 대비: 0:00:00.000531
  └ 전체 경과   : 0:00:44.476644
[13] 배치 종료
  ├ 전 단계 대비: 0:00:00.007986
  └ 전체 경

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 91247093-7569-4906-ac20-e02113725348
.[01] 배치 시작 - [정정공고]물금2 천년나무(17형) 행복주택 입주자격 완화, 선착순 동호지정 모집 공고
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.007211
  └ 전체 경과   : 0:00:00.007211
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.117335
  └ 전체 경과   : 0:00:00.124546
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.031723
  └ 전체 경과   : 0:00:00.156269
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000012
  └ 전체 경과   : 0:00:00.156281
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.401856
  └ 전체 경과   : 0:00:00.558137
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.023040
  └ 전체 경과   : 0:00:00.581177
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:01:18.152773
  └ 전체 경과   : 0:01:18.733950
[09] Markdown -> 청크
  ├ 전 단계 대비: 0:00:00.012098
  └ 전체 경과   : 0:01:18.746048
[10] 청크 -> 임베딩
  ├ 전 단계 대비: 0:00:05.476021
  └ 전체 경과   : 0:01:24.222069
[11] 임베딩 정보 기록
  ├ 전 단계 대비: 0:00:00.349466
  └ 전체 경과   : 0:01:24.571535
[12] 파일 삭제
  ├ 전 단계 대비: 0:00:00.000135
  └ 전체 경과   : 0:01:24.571670
[13] 배치 종료
  ├ 전 단계

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Started parsing the file under job_id 1c789a5d-ecfd-40a3-aeb5-775a3bbd6ed5
[01] 배치 시작 - [정정공고]광주광역시(북구,광산구) 고령자 매입임대주택 예비입주자 모집
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.007477
  └ 전체 경과   : 0:00:00.007477
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.130553
  └ 전체 경과   : 0:00:00.138030
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.032672
  └ 전체 경과   : 0:00:00.170702
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000012
  └ 전체 경과   : 0:00:00.170714
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.455488
  └ 전체 경과   : 0:00:00.626202
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.021539
  └ 전체 경과   : 0:00:00.647741
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:01:06.512413
  └ 전체 경과   : 0:01:07.160154
[09] Markdown -> 청크
  ├ 전 단계 대비: 0:00:00.016971
  └ 전체 경과   : 0:01:07.177125
[10] 청크 -> 임베딩
  ├ 전 단계 대비: 0:00:04.556034
  └ 전체 경과   : 0:01:11.733159
[11] 임베딩 정보 기록
  ├ 전 단계 대비: 0:00:00.311917
  └ 전체 경과   : 0:01:12.045076
[12] 파일 삭제
  ├ 전 단계 대비: 0:00:00.000982
  └ 전체 경과   : 0:01:12.046058
[13] 배치 종료
  ├ 전 단계 대비: 0:00:

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 534f0e95-b8cb-4af2-bef3-d41b31626f4c
[01] 배치 시작 - 군산지역 국민임대주택 입주자 상시 모집(산북부향2차 외 4개단지)
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.007230
  └ 전체 경과   : 0:00:00.007230
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.187294
  └ 전체 경과   : 0:00:00.194524
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.044922
  └ 전체 경과   : 0:00:00.239446
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000012
  └ 전체 경과   : 0:00:00.239458
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.652253
  └ 전체 경과   : 0:00:00.891711
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.013789
  └ 전체 경과   : 0:00:00.905500
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:01:11.657927
  └ 전체 경과   : 0:01:12.563427
[09] Markdown -> 청크
  ├ 전 단계 대비: 0:00:00.020704
  └ 전체 경과   : 0:01:12.584131
[10] 청크 -> 임베딩
  ├ 전 단계 대비: 0:00:05.405833
  └ 전체 경과   : 0:01:17.989964
[11] 임베딩 정보 기록
  ├ 전 단계 대비: 0:00:00.484765
  └ 전체 경과   : 0:01:18.474729
[12] 파일 삭제
  ├ 전 단계 대비: 0:00:00.000554
  └ 전체 경과   : 0:01:18.475283
[13] 배치 종료
  ├ 전 단계 대비: 0:00:00.

/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.lh.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Started parsing the file under job_id 8f8690cd-a192-47a8-b400-ca8b505f9378
[01] 배치 시작 - 2025 다자녀 전세임대 입주자 수시모집 공고
  ├ 전 단계 대비: 0:00:00
  └ 전체 경과   : 0:00:00
[02] 공고 닫기 처리
  ├ 전 단계 대비: 0:00:00.008957
  └ 전체 경과   : 0:00:00.008957
[03] 파일 조회
  ├ 전 단계 대비: 0:00:00.115805
  └ 전체 경과   : 0:00:00.124762
[04] 파일 기록 삭제 처리
  ├ 전 단계 대비: 0:00:00.022086
  └ 전체 경과   : 0:00:00.146848
[05] 파일 처리 시작 (1/1)
  ├ 전 단계 대비: 0:00:00.000008
  └ 전체 경과   : 0:00:00.146856
[06] 파일 다운로드
  ├ 전 단계 대비: 0:00:00.365365
  └ 전체 경과   : 0:00:00.512221
[07] 파일 정보 DB 기록
  ├ 전 단계 대비: 0:00:00.015917
  └ 전체 경과   : 0:00:00.528138
[08] 파일 -> Markdown
  ├ 전 단계 대비: 0:00:59.872748
  └ 전체 경과   : 0:01:00.400886
[09] Markdown -> 청크
  ├ 전 단계 대비: 0:00:00.004813
  └ 전체 경과   : 0:01:00.405699
[10] 청크 -> 임베딩
  ├ 전 단계 대비: 0:00:01.741041
  └ 전체 경과   : 0:01:02.146740
[11] 임베딩 정보 기록
  ├ 전 단계 대비: 0:00:00.239033
  └ 전체 경과   : 0:01:02.385773
[12] 파일 삭제
  ├ 전 단계 대비: 0:00:00.001630
  └ 전체 경과   : 0:01:02.387403
[13] 배치 종료
  ├ 전 단계 대비: 0:00:00.009038
  └ 